# AI FAQ 챗봇 만들기 with RAG

In [ ]:
# 환경 변수 세팅   

In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema import Document
import json

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

1. 데이터 로드

In [3]:
with open('./data/faq_chatbot_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

data

[{'question': '반품 정책이 어떻게 되나요?',
  'answer': '제품을 수령한 후 14일 이내에 반품이 가능합니다. 반품 신청은 고객센터에서 도와드립니다.'},
 {'question': '배송은 얼마나 걸리나요?',
  'answer': '일반적으로 2~5일 정도 소요되며, 지역에 따라 다를 수 있습니다.'},
 {'question': '회원가입 없이 주문할 수 있나요?',
  'answer': '네, 비회원 주문이 가능합니다. 다만, 주문 조회 및 혜택을 받기 위해 회원가입을 추천드립니다.'},
 {'question': '포인트 적립은 어떻게 되나요?',
  'answer': '구매 금액의 1%가 적립되며, 특별 프로모션 기간 동안 추가 적립이 가능합니다.'},
 {'question': '해외 배송이 가능한가요?',
  'answer': '일부 국가를 대상으로 해외 배송이 가능하며, 배송비는 지역에 따라 상이할 수 있습니다.'},
 {'question': '제품 보증 기간은 얼마나 되나요?',
  'answer': '제품 보증 기간은 기본적으로 1년이며, 일부 제품은 연장 보증이 가능합니다.'},
 {'question': 'A/S 신청은 어떻게 하나요?',
  'answer': '공식 홈페이지 또는 고객센터를 통해 A/S 신청이 가능합니다.'},
 {'question': '재고가 없는 상품은 언제 다시 입고되나요?',
  'answer': '입고 일정은 상품마다 다르며, 알림 설정을 통해 입고 시 안내받을 수 있습니다.'},
 {'question': '구매 후 영수증을 받을 수 있나요?',
  'answer': '네, 주문 내역에서 전자 영수증을 다운로드할 수 있습니다.'},
 {'question': '다른 사람이 대신 주문을 수령할 수 있나요?',
  'answer': '네, 수령인을 지정할 수 있으며 신분증 확인 후 수령 가능합니다.'},
 {'question': '결제 방법에는 어떤 것이 있나요?',
  'answer': '신용카드, 

2. embedding 모델 생성 + vector store 생성

In [11]:
embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [20]:
documents = [
    Document(
        page_content=f"Q: {item['question']}\nA: {item['answer']}"
        # metadata={"source": f"faq_{i}"}
    )
    for i, item in enumerate(data)
]

In [21]:
documents

[Document(metadata={}, page_content='Q: 반품 정책이 어떻게 되나요?\nA: 제품을 수령한 후 14일 이내에 반품이 가능합니다. 반품 신청은 고객센터에서 도와드립니다.'),
 Document(metadata={}, page_content='Q: 배송은 얼마나 걸리나요?\nA: 일반적으로 2~5일 정도 소요되며, 지역에 따라 다를 수 있습니다.'),
 Document(metadata={}, page_content='Q: 회원가입 없이 주문할 수 있나요?\nA: 네, 비회원 주문이 가능합니다. 다만, 주문 조회 및 혜택을 받기 위해 회원가입을 추천드립니다.'),
 Document(metadata={}, page_content='Q: 포인트 적립은 어떻게 되나요?\nA: 구매 금액의 1%가 적립되며, 특별 프로모션 기간 동안 추가 적립이 가능합니다.'),
 Document(metadata={}, page_content='Q: 해외 배송이 가능한가요?\nA: 일부 국가를 대상으로 해외 배송이 가능하며, 배송비는 지역에 따라 상이할 수 있습니다.'),
 Document(metadata={}, page_content='Q: 제품 보증 기간은 얼마나 되나요?\nA: 제품 보증 기간은 기본적으로 1년이며, 일부 제품은 연장 보증이 가능합니다.'),
 Document(metadata={}, page_content='Q: A/S 신청은 어떻게 하나요?\nA: 공식 홈페이지 또는 고객센터를 통해 A/S 신청이 가능합니다.'),
 Document(metadata={}, page_content='Q: 재고가 없는 상품은 언제 다시 입고되나요?\nA: 입고 일정은 상품마다 다르며, 알림 설정을 통해 입고 시 안내받을 수 있습니다.'),
 Document(metadata={}, page_content='Q: 구매 후 영수증을 받을 수 있나요?\nA: 네, 주문 내역에서 전자 영수증을 다운로드할 수 있습니다.'),
 Document(metadata={}, p

In [ ]:
vector_store = Chroma.from_documents(documents, embedding_model)


In [ ]:
# embedding_model = OpenAIEmbeddings()    # 비워주면 기본값으로 들어가는 모델이 있음.

# vector_store = Chroma(
#     documents=documents,
#     embedding_function=embedding_model, 
#     persist_directory='./chroma_db'
# )

# documents = [Document(page_content=item['question']+" "+item['answer']) for item in data]
# vector_store.add_documents(documents)

3. llm 설정 (모델 생성)

In [15]:
model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

4. 프롬프트 및 체인 설정

In [23]:
# PromptTemplate, RetrievalQA

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
너는 고객 지원을 도와주는 AI야. 아래 FAQ 데이터를 참고해서 사용자 질문에 성실하게 답변해줘.
가능하면 정확한 내용을 사용하고, FAQ 내용에서 알 수 없는 정보는 억지로 추측하지 마.

[FAQ 내용]
{context}

[사용자 질문]
{question}

[답변]
"""
)   # prompt를 구성할 때 retriever의 검색 결과로 들어갈 input_variable은 'context'로 설정해야 함

qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=vector_store.as_retriever(),  # 위에서 만든 Chroma
    chain_type_kwargs={"prompt": prompt}
)

5. 사용

In [19]:
result = qa_chain("영수증 보관하는걸 까먹었어요")
print(result['result'])

주문 내역에서 전자 영수증을 다운로드할 수 있으니, 해당 내역을 확인하시면 영수증을 다시 받을 수 있습니다. 도움이 필요하시면 고객센터에 문의해 주세요.


In [24]:
qa_chain.run('영수증 보관하는걸 까먹었어요')

'주문 내역에서 전자 영수증을 다운로드할 수 있으니, 해당 내역을 확인하시면 영수증을 다시 받을 수 있습니다. 도움이 필요하시면 고객센터에 문의해 주세요.'

런팟에서 팟 만들면 웹터미널 들어가는거 있잖아